In [23]:
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from datetime import datetime
from time import sleep as para
from time import time as contar
import datetime as dt

import pandas as pd

from openpyxl import load_workbook, Workbook
from tkinter import messagebox
from docx import Document
import win32com.client
from tkinter import Tk
import os
import re

### Doc_Contratos

Bloco já executado e funcional. As planilhas de contratos estão na pasta: \\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS

In [ ]:
class Doc_Contratos():
    def __init__(self):
        for ano in range(2015,2023):
            # Exemplo de uso
            save = os.path.join(r'\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS', f'contratos de obras - {ano}.xlsx')
            caminho_pasta = os.path.join(r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS", f"CONTRATO_{ano}")
            if os.path.exists(save): print(f"Arquivo contratos de obras - {ano}.xlsx já existe!")
            else:
                df = self.ler_arquivos_em_pasta(caminho_pasta, ano)
                info = pd.DataFrame(df).transpose()
                info.to_excel(save, index=True)

    def ler_arquivo_docx(self, caminho_arquivo):
        print('+++++++++++ Executando ler_arquivo_docx! +++++++++++')
        doc = Document(caminho_arquivo)
        
        conteudo = ""
        for paragrafo in doc.paragraphs:
            conteudo += paragrafo.text + "\n"
        
        return conteudo

    def ler_arquivo_doc(self, caminho_arquivo_doc, word):
        print('+++++++++++ Executando ler_arquivo_doc! +++++++++++')
        doc = word.Documents.Open(caminho_arquivo_doc)
        
        # Extrai o texto do documento .doc
        texto = doc.Content.Text
        
        # Fecha o documento e o aplicativo Word
        doc.Close()
        
        return texto

    def extraindo_informacoes(self, texto, arquivo):
        # print(texto.replace('\r','\n'))
        print(f'+++++++++++ Executando extraindo_informacoes! {arquivo} +++++++++++')
        # Extrair número do contrato
        # numero_contrato = re.search(r"Contrato nº\.\s+(\d+).*\r", texto).group(1)
        numero_contrato = re.search(r"Contrato n[º|°]\.?\s*(\d+).*\r", texto).group(1)

        # Extrair data do contrato
        data_contrato = re.search(r"celebrado [em|em\s|\s]*(\d{1,2}.?[/|\.]\d{1,2}[/|\.]\d{4})", texto).group(1)
        # data_contrato = datetime.strptime(data_contrato, "%d.%m.%Y")

        # Extrair nome da empresa
        # nome_empresa = re.search(r"[empresa\s+|consórcio](.+)[\r]?[3.OBJETO]?", texto).group(1)
        # nome_empresa = re.search(r"PARTES: (.+)\r", texto).group(1)
        nome_empresa = re.search(r"PARTES:\s*(.+)\r", texto).group(1)

        # Extrair objeto/obra
        # objeto_obra = re.search(r"OBJETO:\s(.+), [de acordo|conformeManaus/AM]", texto).group(1)
        # objeto_obra = re.search(r"OBJETO:\s*(.+), [de acordo|conforme|Manaus/AM]", texto).group(1)
        objeto_obra = re.search(r"OBJETO[:]?\s*(.+), [de acordo|conformeManaus/AM]", texto).group(1)

        # Extrair valor global
        valor_global = re.search(r"[R\$|VALOR GLOBAL:] ([\d.,]+)", texto).group(1)

        # Extrair nota de empenho
        nota_empenho = re.findall(r"[nº|n|n°][\.|\s]+(\b\d{5}\b)", texto)
        'Empenhos nº. 01136 e nº 01138,'

        # Extrair prazo
        # prazo = re.search(r"PRAZO:\D+(\d+)\D+", texto).group(1)

        # Imprimir as informações extraídas
        # print("Número do contrato:", numero_contrato)
        # print("Data do contrato:", data_contrato)
        # print("Empresa:", nome_empresa)
        # print("Objeto/Obra:", objeto_obra)
        # print("Valor global:", valor_global)
        # print("Nota de empenho:", nota_empenho)
        # print("Prazo:", prazo)

        return (numero_contrato, data_contrato, nome_empresa, objeto_obra, valor_global, tuple(nota_empenho))

    def ler_arquivos_em_pasta(self, caminho_pasta, ano):
        print(f'+++++++++++ Extraindo ler_arquivos_em_pasta! ({ano}) +++++++++++')
        df = {}
        arquivos = os.listdir(caminho_pasta)
        word = win32com.client.Dispatch("Word.Application")
        
        # print('Extraindo ler_arquivos_em_pasta!')
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(caminho_pasta, arquivo)
            
            if '.doc' in arquivo and os.path.isfile(caminho_arquivo): texto = self.ler_arquivo_doc(caminho_arquivo, word)
            else: texto = self.ler_arquivo_docx(caminho_arquivo=caminho_arquivo)

            informacao = self.extraindo_informacoes(texto, arquivo)
            df[arquivo] = informacao
        
        word.Quit()

        return df

Doc_Contratos()

### Detalhando NEs

In [2]:
class Navegacao():
    def __init__(self):

        servico = Service(ChromeDriverManager().install())

        print("Fechar a janela!")
        ask = messagebox.askyesno("Esconder Navegador", "Deseja esconder o navegador?")
        janela = Tk()
        if ask:
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            navegador = webdriver.Chrome(options=options, service=servico)
        else: navegador = webdriver.Chrome(service=servico)
        janela.destroy()
        print("Janela fechada!")
        self.vars = {'ignored_exceptions': (NoSuchElementException,StaleElementReferenceException), 
                    'senha': f'Jaspekde27.{dt.date.today().month*3}'}
        self.navegador = navegador

    def esperar_elementos_carregarem(navegador, tipo_elemento: str, nome_elemento: str, varios_elementos: bool = False, tempo_maximo_espera: int = 30):
        """Espera um ou vários elementos carregarem na página

        Args:
            tipo_elemento (str): tipo do elemento
            nome_elemento (str): nome do elemento
            varios_elementos (bool): procurar mais de um elemento
            tempo_maximo_espera (int): tempo determinado para executar a espera, em segundos. Padrão 30 (0,5 min).
        """    
        tempo_inicial = contar() 
        
        if varios_elementos:
            elementos = WebDriverWait(navegador, tempo_maximo_espera, ignored_exceptions=(NoSuchElementException,StaleElementReferenceException))\
                            .until(expected_conditions.presence_of_all_elements_located((tipo_elemento, nome_elemento)))
        else:
            elemento = WebDriverWait(navegador, tempo_maximo_espera, ignored_exceptions=(NoSuchElementException,StaleElementReferenceException))\
                            .until(expected_conditions.presence_of_element_located((tipo_elemento, nome_elemento)))
            
        tempo_final = contar()

        if varios_elementos:
            print(f'>>> Elementos "{nome_elemento}" encontrados em {round(tempo_final - tempo_inicial, 2)}s! (len({len(elementos)}))')
            return elementos
        else:
            if len(elemento.text) > 20:
                elemento_texto_resumido = elemento.text[:20]
            else:
                elemento_texto_resumido = elemento.text
                
            print(f'>>> Elemento "{elemento_texto_resumido}" encontrado em {round(tempo_final - tempo_inicial, 2)}s!')
            return elemento
        
    def LISNE(self, NE, afim):
        print('\n=============== Executando LISNE! ===============')
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        if 'NE' in NE: NE = NE.split("NE")[1]
        lisne_link = f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/' + '/Lisne.do'
        self.navegador.get(lisne_link)

        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.navegador.find_element(By.NAME, 'cboUnidadeGestora').send_keys('270101 - Secretaria Municipal de Infraestrutura')
        self.navegador.find_element(By.ID, 'mkdNumeroNE').clear()
        self.navegador.find_element(By.ID, 'mkdNumeroNE').send_keys(NE)
        try: self.navegador.find_element(By.ID, 'btnProcurar').click()
        except: self.navegador.find_element(By.CSS_SELECTOR, '#IMGprocurar').click()

        para(2)
        try: 
            elemento = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'tr td.readCell')
            ActionChains(self.navegador).double_click(elemento).perform()

        except:
            self.esperar_elementos_carregarem(self.navegador, By.CSS_SELECTOR, '[class="btn btn-outline-primary waves-effect waves-light"]').click()

        para(2)
        # navegador.switch_to.window(navegador.window_handles[-1])

        fonte_de_recurso = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtFonte"]').get_attribute('value')
        natureza_despesa = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtNaturezaDespesa"]').get_attribute('value')
        empenho_original = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtNEOriginal"]').get_attribute('value')
        processo = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtProcesso"]').get_attribute('value')
        valor = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtValor"]').get_attribute('value')

        self.navegador.find_element(By.CSS_SELECTOR, '[data-dismiss="modal-empenho"]').click()

        return (NE, fonte_de_recurso, natureza_despesa, empenho_original, processo, valor)
    
    def NL_detalhada(self, obj_NL):
        """Pesquisa a Nota de Lançamento desejada para retirar as informações

        Args:
            obj_NL (_type_): Objeto Selenium da Nota de Lançamento

        Returns:
            _type_: Tupla com as informações extraidas
        """
        if not 'NL' in obj_NL.text: 
            print(f'\n=============== Impossível executar! Não é uma NL ({obj_NL.text}) ===============')
            return ('', '', '', '', '', '','')
        
        
        print(f'\n=============== Executando a NL ({obj_NL.text}) ===============')
        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        elemento = obj_NL
        ActionChains(self.navegador)\
        .double_click(elemento)\
        .perform()

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        credor_empresa = self.esperar_elemento(By.CSS_SELECTOR, 'tr td input[name="txtCredor"]').get_attribute('value')
        obs = self.esperar_elemento(By.CSS_SELECTOR, 'textarea[name="txtObservacao"]').text
        NE = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtNuNe"').get_attribute('value')
        natureza_despesa = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtRdNaturezaDespesa"]').get_attribute('value')
        doc = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtRdNuDocmento"').get_attribute('value')
        processo = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtRdNuProcesso"').get_attribute('value')
        
        self.navegador.close()
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        return (obj_NL.text, credor_empresa, obs, NE, natureza_despesa, doc, processo)
    
    def entra_AFIM(self, afim):
        """Abre o AFIM

        Args:
            afim (_type_): Ano no qual o FIM irá abrir
        """
        print(f'\n=============== Executando entra_AFIM {afim}! ===============')
        
        if afim <= 2018:
            self.navegador.get(f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/')
            self.navegador.find_element(By.NAME, 'user').send_keys('03441042220')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars["senha"])
            self.navegador.find_element(By.NAME, 'enviar').click()
            para(1)
            self.navegador.switch_to.window(self.navegador.window_handles[-1])
            # self.navegador.switch_to.frame(1)

        #AFIM DE 2019
        if afim == 2019:
            self.navegador.get('https://afim1.manaus.am.gov.br/AfimPRD2019/')
            self.navegador.find_element(By.NAME, 'user').send_keys('034.410.422-20')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars["senha"])
            
            para(2)
            for w in self.navegador.window_handles:
                self.navegador.switch_to.window(w)
                if 'mensagem' in self.navegador.current_url: self.navegador.close()
                elif str(afim) in self.navegador.current_url: pass
                else: self.navegador.close()
                
            self.navegador.switch_to.window(self.navegador.window_handles[-1])
            # self.navegador.switch_to.frame(1)

        if afim >= 2020:
            self.navegador.get(f'https://afim1.manaus.am.gov.br/AfimPRD{afim}/')
            self.navegador.find_element(By.NAME, 'user').send_keys('034.410.422-20')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars["senha"])
            
            self.navegador.switch_to.window(self.navegador.window_handles[-1])
            para(1)
            # self.navegador.switch_to.frame(0)

        # try: self.navegador.find_element(By.CSS_SELECTOR, "body").click()
        # except: self.esperar_elementos_carregarem(By.CSS_SELECTOR, "body").click()

        print(f'=============== Finalizando entra_AFIM {afim}! ===============\n')
        para(2)

    def RELSALEMP(self, NE, afim):
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        if 'NE' in NE: NE = NE.split("NE")[1]
        lisne_link = f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/' + '/Relsalemp.do'
        self.navegador.get(lisne_link)

        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.navegador.find_element(By.CSS_SELECTOR, 'option[value="270101"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, '[name="numeroNE"]').send_keys(NE)
        self.navegador.find_element(By.CSS_SELECTOR, '#IMGimprimir').click()

        while len([f for f in os.listdir(self._vars['path']) if os.path.isfile(os.path.join(self._vars['path'], f)) and (('' in f) or ('RelBalanceteMensal.xls' in f))]) > 1: 
            self._mensagem('Apague, renomeie ou mova os arquivos', 'Entre na pasta de Downloads e apague ou renomeie os arquivos "doc.pdf" e "RelBalanceteMensal.xls", para que o programa possa executar sem erros!')



In [ ]:
ano = 2014

driver = Navegacao()
driver.entra_AFIM(ano)
driver.LISNE('01136',ano)

In [ ]:
NE = '01136'

driver.navegador.switch_to.window(driver.navegador.window_handles[-1])
if 'NE' in NE: NE = NE.split("NE")[1]
lisne_link = f'http://afim2.manaus.am.gov.br/AfimPRD{ano}/' + '/Relsalemp.do'
driver.navegador.get(lisne_link)

In [ ]:
driver.navegador.find_element(By.CSS_SELECTOR, 'option[value="270101"]').click()
driver.navegador.find_element(By.CSS_SELECTOR, '[name="numeroNE"]').send_keys(NE)
driver.navegador.find_element(By.CSS_SELECTOR, '#IMGimprimir').click()
